In [35]:
import regex

from pathlib import Path

In [37]:
def find_occurences(pattern: str, limit: int = None):
    out = {}
    for ustawa in list(Path("./ustawy/").iterdir())[:limit]:
        found = regex.findall(pattern=pattern, string=ustawa.read_text(encoding="utf8"))
        out[ustawa.name] = found
    return out

In [73]:
# Przydatny regex, dopasowujący wystąpienie jednostki redakcyjnej 
# artykuł, paragraf, ustęp, punkt, litera
# wraz z zakresem (np. art. 1-3)
# grupy są non-capturing, żeby regex nie zwracał krotek

jedn_regex = r"(?:art\. \d+\w*(?:-\d+\w*)*|§ \d+(?:-\d+)*|ust\. \d+\w*(?:-\d+\w*)*|pkt \d+\w*(?:-\d+\w*)*|lit\.\w+\)(?:-\w+\))*)"

find_occurences(jedn_regex, 1)["1993_599.txt"][32:42]

['ust. 2a',
 'ust. 2',
 'ust. 3',
 'ust. 1',
 'art. 14',
 'art. 7',
 'ust. 1',
 'art. 47',
 'art. 21',
 'pkt 2-4']

### 1. Dla każdej ustawy policz wszystkie wystąpienie następujących nowelizacji:

#### a. dodanie jednostki (e.g. dodaje się ust. 5a)

Struktura:
- na początku dodania występuje równoważnik zdania zawierający synonim "dodaje się" (składa się z liter, liczb, kropek, spacji, § - bo mogą być wypisane jednostki redakcyjne),
- następnie określona jest dodawana jednostka redakcyjna (lub jednostki),
- na końcu występuje "w brzmieniu:".

In [110]:
r1a = fr"[\w\d\.§ ]+{jedn_regex}(?: i {jedn_regex}| i \d+\w*)? w brzmieniu:"
out1a = find_occurences(pattern=r1a, limit=10)
{k: len(v) for k, v in out1a.items()}

{'1993_599.txt': 10,
 '1993_602.txt': 1,
 '1993_645.txt': 0,
 '1993_646.txt': 18,
 '1994_150.txt': 0,
 '1994_195.txt': 2,
 '1994_201.txt': 1,
 '1994_214.txt': 0,
 '1994_215.txt': 1,
 '1994_288.txt': 0}

#### b. usunięcie jednostki (np. w art. 10 ust. 1 pkt 8 skreśla się)

Struktura:
- w skreśleniu występuje "skreśla się",
- następnie występuje usuwana jednostka redakcyjna.

Nie chcemy wykryć usunięcia czegoś innego niż jednostki redakcyjnej (np. wyrazy).

In [123]:
r1b = fr"skreśla się {jedn_regex}"
out1b = find_occurences(pattern=r1b, limit=10)
{k: len(v) for k, v in out1b.items()}

{'1993_599.txt': 1,
 '1993_602.txt': 0,
 '1993_645.txt': 0,
 '1993_646.txt': 4,
 '1994_150.txt': 1,
 '1994_195.txt': 1,
 '1994_201.txt': 0,
 '1994_214.txt': 0,
 '1994_215.txt': 0,
 '1994_288.txt': 0}

#### c. zmianę jednostki (np. art. 5 otrzymuje brzmienie)

Struktura:
- zmiana jednostki zaczyna się od nazwy jednostki (lub jednostek),
- następnie występuje "otrzymuje/ą brzmienie:".

In [127]:
r1c = fr"{jedn_regex}(?: i {jedn_regex}| i \d+\w*)? otrzymuj[eą] brzmienie:"
out1c = find_occurences(pattern=r1c, limit=10)
{k: len(v) for k, v in out1c.items()}

{'1993_599.txt': 17,
 '1993_602.txt': 8,
 '1993_645.txt': 0,
 '1993_646.txt': 35,
 '1994_150.txt': 1,
 '1994_195.txt': 1,
 '1994_201.txt': 0,
 '1994_214.txt': 0,
 '1994_215.txt': 2,
 '1994_288.txt': 0}